<a href="https://colab.research.google.com/github/logansdmi/CurveFitting/blob/main/CurveFitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports relevant libraries
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.style 
import matplotlib as mpl 


def polyModelWithNoise(x,theta,sigma):
  